In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('Missing').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
training=spark.read.csv('/home/ubuntu/pyspark/data/Tutorial_6/SalaryData .csv',  header=True, inferSchema=True, nullValue="NULL")

In [3]:
training.show()

+------+---+----------+------+
|  Name|Age|Experience|Salary|
+------+---+----------+------+
| Sunny| 52|        10| 30000|
|   May| 19|         5| 20000|
|   Bob| 35|         4| 40000|
|  Paul| 42|         2| 15000|
|Mahesh| 36|         4| 50000|
| Masha| 35|         5| 20000|
+------+---+----------+------+



In [4]:
training.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [5]:
training.columns

['Name', 'Age', 'Experience', 'Salary']

In [6]:
from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["Age","Experience"],outputCol="Independent Features")

In [7]:
output=featureassembler.transform(training)

In [8]:
output.show()

+------+---+----------+------+--------------------+
|  Name|Age|Experience|Salary|Independent Features|
+------+---+----------+------+--------------------+
| Sunny| 52|        10| 30000|         [52.0,10.0]|
|   May| 19|         5| 20000|          [19.0,5.0]|
|   Bob| 35|         4| 40000|          [35.0,4.0]|
|  Paul| 42|         2| 15000|          [42.0,2.0]|
|Mahesh| 36|         4| 50000|          [36.0,4.0]|
| Masha| 35|         5| 20000|          [35.0,5.0]|
+------+---+----------+------+--------------------+



In [9]:
output.columns

['Name', 'Age', 'Experience', 'Salary', 'Independent Features']

In [10]:
finalized_data=output.select("Independent Features","Salary")

In [11]:
finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [52.0,10.0]| 30000|
|          [19.0,5.0]| 20000|
|          [35.0,4.0]| 40000|
|          [42.0,2.0]| 15000|
|          [36.0,4.0]| 50000|
|          [35.0,5.0]| 20000|
+--------------------+------+



In [12]:
from pyspark.ml.regression import LinearRegression
##traintest split
train_data,test_data=finalized_data.randomSplit([0.75,0.25])
regressor=LinearRegression(featuresCol="Independent Features",labelCol="Salary")
regressor=regressor.fit(train_data)

In [13]:
### Coefficents
regressor.coefficients

DenseVector([49.5959, 1060.4947])

In [14]:
### Intercepts
regressor.intercept

17670.217976977736

In [15]:
test_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|          [36.0,4.0]| 50000|
+--------------------+------+



In [16]:
### Prediction
pred_results=regressor.evaluate(test_data)

In [17]:
pred_results.predictions.show()

+--------------------+------+-----------------+
|Independent Features|Salary|       prediction|
+--------------------+------+-----------------+
|          [36.0,4.0]| 50000|23697.64878765613|
+--------------------+------+-----------------+

